In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import foolbox as fb
import foolbox.attacks as fa
from foolbox import PyTorchModel, accuracy, samples

import numpy as np

import statistics

In [3]:
n_epochs = 1
batchsize = 64
learning_rate = 0.001
momentum = 0.9
log_interval = 100

In [4]:
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.ToTensor(),
     torchvision.transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchsize,
                                          shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batchsize,
                                         shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729141890/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5,stride=1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(120,84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
network = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=learning_rate)       

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

print(" ")
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()        

 
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302818
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.293363
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.219899
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.080069
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.188698
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.149624
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.040187
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.187087
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.067371
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.074860

Test set: Avg. loss: 0.0013, Accuracy: 9717/10000 (97%)



In [7]:
#This function does the same work as foolbox, it allows us to compare the values of foolbox
#It's only used to check if there is no issues with the model given to foolbox

def benchmark():   
    listeimg = []
    listeadv = []
    for i in range(len(images)):
      imaj = images[i]
      img = imaj
      listeimg.append(img)
      tempadvlist = []
      for e in range(len(epsilons)):
        iadvs = advs[e][i]
        adv = iadvs
        tempadvlist.append(adv)
      listeadv.append(tempadvlist)

    results = []

    model.eval()
    for e in range(len(epsilons)):
      tempepsilons = []
      for i in range(len(listeadv)):
        imgadvtest = listeadv[i][e]
        lb = labels[i].item()
        with torch.no_grad():
          output = model(imgadvtest.unsqueeze(0))
        pred = output.data.max(1, keepdim=True)[1][0].item() 
        prob = torch.nn.functional.softmax(output, dim=1)
        top_p, top_class = prob.topk(1, dim = 1)
        if(str(lb)==str(pred)):
          tempepsilons.append("FALSE")
        else:
          tempepsilons.append("TRUE")
      results.append(tempepsilons)

    for i in range(len(results)):
        v = 0
        for e in range(len(results[0])):
            if results[i][e] == "FALSE":
                v = v+1
        print("eps: "+ str(epsilons[i]), end=' | ')
        print("pred en %: " + str(float(v/len(results[0]))*100)) 
        print("")

In [8]:
adv_loader = torch.utils.data.DataLoader(testset, batch_size=batchsize,
                                         shuffle=False)

examples = enumerate(adv_loader)
batch_idx, (images, labels) = next(examples)

In [9]:
model = network
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

print(" ")
print("accuracy", end=' | ')
print(accuracy(fmodel, images, labels))
print("")

attacks = [fb.attacks.PGD()]

epsilons = [
    0.0,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    1.0,
    1.1,
    1.2,
    1.3,
    1.4,
    1.5,
    1.6,
    1.7,
    1.8,
    1.9,
    2.0,
]

print("epsilons")
print(epsilons)
print("")


 
accuracy | 0.984375

epsilons
[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]



In [11]:
lbyat = []
fulatac = []
for i, attack in enumerate(attacks):
    _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    benchmark()
    atacc = []
    robust_accuracy = 1 - success.float().mean(axis=-1)
    for eps, acc in zip(epsilons, robust_accuracy):
        print(attack, eps, acc.item())
        atacc.append(acc.item())
    print(" ")
    fulatac.append(atacc)
    lfull = []
    for e in range(len(epsilons)):
        l = []
        for i in range(len(images)):
            perturbation = advs[e][i][0].numpy() - images[i][0].numpy()
            l.append(float(format(np.linalg.norm(perturbation.flatten()))))
        lfull.append(statistics.mean(l))
    lbyat.append(lfull)

eps: 0.0 | pred en %: 98.4375

eps: 0.1 | pred en %: 98.4375

eps: 0.2 | pred en %: 96.875

eps: 0.3 | pred en %: 96.875

eps: 0.4 | pred en %: 93.75

eps: 0.5 | pred en %: 87.5

eps: 0.6 | pred en %: 70.3125

eps: 0.7 | pred en %: 35.9375

eps: 0.8 | pred en %: 9.375

eps: 0.9 | pred en %: 7.8125

eps: 1.0 | pred en %: 3.125

eps: 1.1 | pred en %: 1.5625

eps: 1.2 | pred en %: 1.5625

eps: 1.3 | pred en %: 0.0

eps: 1.4 | pred en %: 0.0

eps: 1.5 | pred en %: 0.0

eps: 1.6 | pred en %: 0.0

eps: 1.7 | pred en %: 0.0

eps: 1.8 | pred en %: 0.0

eps: 1.9 | pred en %: 0.0

eps: 2.0 | pred en %: 0.0

LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True) 0.0 0.984375
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True) 0.1 0.984375
LinfProjectedGradientDescentAttack(rel_stepsize=0.03333333333333333, abs_stepsize=None, steps=40, random_start=True) 0.2 0.96875
LinfPr